In [13]:
import tkinter as tk
from tkinter import ttk, simpledialog, messagebox
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.patheffects as path_effects
import unicodedata
from flask import Flask, render_template, request
import os

In [14]:
# Cargar los archivos Excel
file_path_cbl = 'Tarifas_CBL.xlsx'
file_path_ontime = 'Tarifas_ONTIME.xlsx'
file_path_mrw = 'Tarifas_MRW.xlsx'
file_path_productos = 'Productos.xlsx'

In [15]:
# Leer los archivos en Pandas
df_cbl = pd.read_excel(file_path_cbl)
df_ontime = pd.read_excel(file_path_ontime)
df_mrw = pd.read_excel(file_path_mrw)
df_productos_devol = pd.read_excel(file_path_productos)

In [16]:
# Ruta del archivo ShapeFile con los límites provinciales
shapefile_path = 'C:/Users/Svan/OneDrive - SVAN TRADING SL/Documentos/Proyectos/Calculadora transporte/lineas_limite/SHP_ETRS89/ll_provinciales_inspire_peninbal_etrs89/ll_provinciales_inspire_peninbal_etrs89.shp'

In [17]:
gdf = gpd.read_file(shapefile_path)

DataSourceError: C:/Users/Svan/OneDrive - SVAN TRADING SL/Documentos/Proyectos/Calculadora transporte/lineas_limite/SHP_ETRS89/ll_provinciales_inspire_peninbal_etrs89/ll_provinciales_inspire_peninbal_etrs89.shp: No such file or directory

In [ ]:
provincias_mrw = [
    "VALENCIA", "ALBACETE", "ALICANTE", "CASTELLON", "CUENCA", "BARCELONA", "TARRAGONA", "MADRID", "MURCIA", 
    "ALMERIA", "ZARAGOZA", "GUADALAJARA", "TOLEDO", "GIRONA", "LLEIDA", "CORDOBA", "GRANADA", "SEVILLA", 
    "JAEN", "CIUDAD REAL", "BURGOS", "SEGOVIA", "VALLADOLID", "LA RIOJA", "NAVARRA", "VIZCAYA", "CADIZ", 
    "MALAGA", "HUESCA", "ASTURIAS", "CANTABRIA", "AVILA", "LEON", "BADAJOZ", "CACERES", "VIZCAYA", "GUIPUZKOA", 
    "HUELVA", "TERUEL", "PALENCIA", "SALAMANCA", "SORIA", "ZAMORA", "A CORUÑA", "LUGO", "ORENSE", "PONTEVEDRA", 
    "MALLORCA", "IBIZA", "MENORCA"
]

provincias_cbl = provincias_mrw

provincias_ontime = ["A CORUÑA", "ALAVA", "ALBACETE", "ALICANTE", "ALMERIA", "ASTURIAS", "AVILA", "BADAJOZ", "PALMA DE MALLORCA", 
    "MENORCA", "BARCELONA", "BURGOS", "CACERES", "CADIZ", "CANTABRIA", "CASTELLON", "CIUDAD REAL", "CORDOBA", 
    "CUENCA", "GUIPUZKOA", "GIRONA", "GRANADA", "GRAN CANARIA", "GUADALAJARA", "HUELVA", "HUESCA", "JAEN", 
    "LA RIOJA", "LANZAROTE", "LEON", "LLEIDA", "LUGO", "MADRID", "MALAGA", "MURCIA", "NAVARRA", "ORENSE", 
    "PALENCIA", "PONTEVEDRA", "SALAMANCA", "SEGOVIA", "SEVILLA", "SORIA", "TARRAGONA", "TERUEL", "TOLEDO", 
    "VALENCIA", "VALLADOLID", "VIZCAYA", "ZAMORA", "ZARAGOZA", "PORTUGAL LISBOA", "PORTUGAL OPORTO", 
    "PORTUGAL COIMBRA", "PORTUGAL ZONA2", "GIBRALTAR", "CEUTA", "MELILLA", "ANDORRA"
]

provincias_unificadas = list(set(provincias_mrw + provincias_cbl + provincias_ontime))
provincias_unificadas.sort() # Ordenar alfabeticamente

In [ ]:
# Lista de provincias unificadas
provincias_unificadas = [
    'A CORUÑA', 'ALAVA', 'ALBACETE', 'ALICANTE', 'ALMERIA', 'ASTURIAS', 'AVILA',
    'BADAJOZ', 'PALMA DE MALLORCA', 'MENORCA', 'BARCELONA', 'BURGOS', 'CACERES',
    'CADIZ', 'CANTABRIA', 'CASTELLON', 'CIUDAD REAL', 'CORDOBA', 'CUENCA',
    'GUIPUZKOA', 'GIRONA', 'GRANADA', 'GRAN CANARIA', 'GUADALAJARA', 'HUELVA',
    'HUESCA', 'JAEN', 'LA RIOJA', 'LANZAROTE', 'LEON', 'LLEIDA', 'LUGO',
    'MADRID', 'MALAGA', 'MURCIA', 'NAVARRA', 'ORENSE', 'PALENCIA', 'PONTEVEDRA',
    'SALAMANCA', 'SEGOVIA', 'SEVILLA', 'SORIA', 'TARRAGONA', 'TERUEL', 'TOLEDO',
    'VALENCIA', 'VALLADOLID', 'VIZCAYA', 'ZAMORA', 'ZARAGOZA', 'PORTUGAL LISBOA',
    'PORTUGAL OPORTO', 'PORTUGAL COIMBRA', 'PORTUGAL ZONA2', 'GIBRALTAR', 'CEUTA',
    'MELILLA', 'ANDORRA'
]

# Mapeo de provincias a zonas (ahora se define explícitamente)
provincias_zonas = {provincia: provincia for provincia in provincias_unificadas}

# Función para normalizar y ajustar nombres de provincias
def normalizar_provincia(nombre_provincia):
    nombre_normalizado = ''.join(
        (c for c in unicodedata.normalize('NFD', nombre_provincia) if unicodedata.category(c) != 'Mn')
    ).upper()
    return nombre_normalizado

def ajustar_nombres_provincias(df, columna_provincia='PROVINCIA DESTINO'):
    if columna_provincia in df.columns:
        df[columna_provincia] = df[columna_provincia].apply(normalizar_provincia)
    else:
        print(f"La columna '{columna_provincia}' no se encuentra en el DataFrame.")
    return df

def obtener_tarifa_ontime(df, zona, peso):
    print(f"Buscando tarifa ONTIME para zona: '{zona}', peso: {peso}")
    tarifas_ontime = [int(col) for col in df.columns if col.isdigit()]
    closest_weight_col = min((x for x in tarifas_ontime if x >= peso), default=None)
    
    if closest_weight_col is not None:
        tarifa_ontime_row = df[df['PROVINCIA DESTINO'] == zona]
        if not tarifa_ontime_row.empty:
            tarifa = tarifa_ontime_row.iloc[0][str(closest_weight_col)]
            print(f"Tarifa ONTIME encontrada: {tarifa}")
            return tarifa
    
    print("No se encontró tarifa ONTIME válida.")
    return np.nan

def obtener_tarifa_ontime_xs(df, zona, peso, modalidad):
    try:
        print(f"Buscando tarifa ONTIME XS para zona: '{zona}', peso: {peso}, modalidad: {modalidad}")
        tarifas_ontime = [col for col in df.columns if modalidad in col]
        closest_weight_col = None

        for col in tarifas_ontime:
            weight_limit = int(col.split()[0])
            if weight_limit >= peso:
                closest_weight_col = col
                break
        
        if closest_weight_col is not None:
            tarifa_ontime_row = df[df['PROVINCIA DESTINO'] == zona]
            if not tarifa_ontime_row.empty:
                tarifa = tarifa_ontime_row.iloc[0][closest_weight_col]
                print(f"Tarifa ONTIME XS encontrada: {tarifa}")
                return tarifa

        print(f"No se encontró tarifa ONTIME XS válida para {zona} con peso {peso}")
        return np.nan
    
    except Exception as e:
        print(f'Error en obtener_tarifa_ontime_xs: {e}')
        return np.nan

def obtener_tarifa_mrw(df, zona, peso_total, num_bultos):
    print(f"Buscando tarifa MRW para zona: '{zona}', peso total: {peso_total}, número de bultos: {num_bultos}")
    filtered_df = df[df['KG'] >= peso_total]
    if filtered_df.empty:
        print("No se encontró tarifa MRW válida.")
        return np.nan, np.nan, np.nan
    closest_weight_row = filtered_df.iloc[0]
    tarifa_base = closest_weight_row[zona] if zona in df.columns else np.nan
    recargo_bultos = max(0, num_bultos - 2) * 2
    tarifa_total = tarifa_base + recargo_bultos
    print(f"Tarifa MRW encontrada: base {tarifa_base}, recargo {recargo_bultos}, total {tarifa_total}")
    return tarifa_base, recargo_bultos, tarifa_total

def obtener_tarifa_cbl(df, zona, peso):
    try:
        print(f"Buscando tarifa CBL para zona: '{zona}', peso: {peso}")
        
        if zona not in df.columns:
            print(f"Error: La zona '{zona}' no está en las columnas del DataFrame")
            print(f"Columnas disponibles: {df.columns.tolist()}")
            return np.nan
        
        filtered_df = df[df['KG'] >= peso]
        if filtered_df.empty:
            print(f"No se encontró tarifa CBL para el peso: {peso}")
            return np.nan
        
        closest_weight_row = filtered_df.iloc[0]
        tarifa_cbl = closest_weight_row[zona]
        print(f"Tarifa CBL encontrada para {zona}: {tarifa_cbl}")
        return tarifa_cbl
    except Exception as e:
        print(f"Error inesperado al obtener tarifa CBL: {str(e)}")
        return np.nan

class TransportCalculatorApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Calculadora de Transporte")
        self.master.geometry("800x600")

        self.load_data()
        self.create_widgets()

    def load_data(self):
        # Cargar las tarifas específicas desde los archivos correspondientes
        self.df_cbl = pd.read_excel('Tarifas_CBL.xlsx')  # Aquí ajusta el nombre del archivo si es necesario
        self.df_cbl.rename(columns={'KGS': 'KG'}, inplace=True)
        self.df_cbl['KG'] = pd.to_numeric(self.df_cbl['KG'], errors='coerce')

        # Asegurar que los nombres de las columnas en el DataFrame estén normalizados
        self.df_cbl.columns = [normalizar_provincia(col) for col in self.df_cbl.columns]

        self.df_mrw = pd.read_excel('Tarifas_MRW.xlsx')
        self.df_mrw['KG'] = pd.to_numeric(self.df_mrw['KG'], errors='coerce')

        self.df_productos = pd.read_excel('Tarifas_MRW.xlsx', sheet_name='Productos', header=1)

        self.df_ontime = pd.read_excel('Tarifas_ONTIME.xlsx')
        self.df_ontime_xs = pd.read_excel('Tarifas_ONTIME.xlsx', usecols=lambda col: any(x in col for x in ['24', '48', 'PROVINCIA DESTINO']))
        if 'PROVINCIA DESTINO' in self.df_ontime_xs.columns:
            self.df_ontime_xs = ajustar_nombres_provincias(self.df_ontime_xs, 'PROVINCIA DESTINO')
        
        # Cargar los datos de productos
        self.df_productos_devol = pd.read_excel('Productos.xlsx')

    def create_widgets(self):
        self.label_tipo_producto = ttk.Label(self.master, text="Tipo de Producto:")
        self.label_tipo_producto.pack()
        self.tipo_producto = ttk.Combobox(self.master, values=['Normal', 'XS'])
        self.tipo_producto.pack()
        self.tipo_producto.bind("<<ComboboxSelected>>", self.toggle_widgets)

        self.label_palet_type = ttk.Label(self.master, text="Tipo de Palet:")
        self.label_palet_type.pack()
        self.palet_type = ttk.Combobox(self.master, values=["Medio Palet", "Palet Completo"])
        self.palet_type.pack()

        self.label_height = ttk.Label(self.master, text="Altura del Producto:")
        self.label_height.pack()
        self.height = ttk.Combobox(self.master, values=[0.45, 0.6, 0.85, 1.45, 1.6, 1.85, 2.01])
        self.height.pack()

        self.label_province = ttk.Label(self.master, text="Provincia de Destino:")
        self.label_province.pack()
        self.province = ttk.Combobox(self.master, values=list(provincias_zonas.keys()))
        self.province.pack()

        self.label_categoria = ttk.Label(self.master, text='Categoría del Producto:')
        self.label_categoria.pack()
        self.categoria = ttk.Combobox(self.master, values=list(self.df_productos['CATEGORIAS'].unique()))
        self.categoria.pack()

        self.label_sku = ttk.Label(self.master, text='SKU del Producto:')
        self.label_sku.pack()
        self.sku = ttk.Combobox(self.master, values=list(self.df_productos['SKU'].unique()))
        self.sku.pack()

        self.label_modalidad = ttk.Label(self.master, text='Modalidad (horas):')
        self.label_modalidad.pack()
        self.modalidad = ttk.Combobox(self.master, values=['24', '48'])
        self.modalidad.pack()

        self.label_num_bultos = ttk.Label(self.master, text='Número de Bultos:')
        self.label_num_bultos.pack()
        self.num_bultos = ttk.Entry(self.master)
        self.num_bultos.pack()

        button_frame = ttk.Frame(self.master)
        button_frame.pack(pady=10)

        self.calculate_button = ttk.Button(button_frame, text="Calcular", command=self.calculate_tariff)
        self.calculate_button.pack(side=tk.LEFT, padx=5)

        self.map_button = ttk.Button(button_frame, text="Mostrar Mapa", command=self.show_map)
        self.map_button.pack(side=tk.LEFT, padx=5)

        self.return_button = ttk.Button(button_frame, text="Devoluciones", command=self.calculate_return)
        self.return_button.pack(side=tk.LEFT, padx=5)

        self.result_area = tk.Text(self.master, height=20, width=80)
        self.result_area.pack()
        self.result_area.configure(state="disabled")

        self.toggle_widgets()

    def toggle_widgets(self, event=None):
        tipo_producto = self.tipo_producto.get()
        if tipo_producto == 'XS':
            self.palet_type.configure(state='disabled')
            self.height.configure(state='disabled')
            self.categoria.configure(state='normal')
            self.sku.configure(state='normal')
            self.num_bultos.configure(state='normal')
            self.label_modalidad.configure(state='normal')
            self.modalidad.configure(state='normal')
        else:
            self.palet_type.configure(state='normal')
            self.height.configure(state='normal')
            self.categoria.configure(state='disabled')
            self.sku.configure(state='disabled')
            self.num_bultos.configure(state='disabled')
            self.label_modalidad.configure(state='disabled')
            self.modalidad.configure(state='disabled')
            
    def toggle_widgets_devoluciones(self, event=None):
        tipo_producto = self.tipo_producto_devol.get()
        if tipo_producto == 'XS':
            self.palet_type_devol.configure(state='disabled')
            self.height_devol.configure(state='disabled')
            self.categoria_devol.configure(state='normal')
            self.sku_devol.configure(state='normal')
            self.num_bultos_devol.configure(state='normal')
        else:
            self.palet_type_devol.set("Palet Completo")  # Predefinir como "Palet Completo"
            self.palet_type_devol.configure(state='disabled')  # Desactivar la opción
            self.height_devol.set(1.2)  # Altura predeterminada para Palet Completo
            self.height_devol.configure(state='disabled')
            self.categoria_devol.configure(state='disabled')
            self.sku_devol.configure(state='disabled')
            self.num_bultos_devol.configure(state='disabled')

    def calculate_tariff(self):
        tipo_producto = self.tipo_producto.get()
        if tipo_producto == 'XS':
            self.calculate_tariff_xs()
        else:
            self.calculate_tariff_normal()

    def calculate_tariff_xs(self):
        province = self.province.get()
        categoria = self.categoria.get()
        sku = self.sku.get()
        num_bultos = int(self.num_bultos.get()) if self.num_bultos.get() else 0
        modalidad = self.modalidad.get()

        if not province or not num_bultos or not categoria or not sku or not modalidad:
            self.result_area.configure(state="normal")
            self.result_area.insert(tk.END, "Por favor, complete todos los campos.\n")
            self.result_area.configure(state="disabled")
            return

        zona = provincias_zonas.get(province)
        if not zona:
            self.result_area.configure(state="normal")
            self.result_area.insert(tk.END, f"Provincia {province} no reconocida.\n")
            self.result_area.configure(state="disabled")
            return

        # Buscar el peso del producto
        peso_producto = self.df_productos[(self.df_productos['CATEGORIAS'] == categoria) & (self.df_productos['SKU'] == sku)]['PESO BRUTO (kg)'].values
        if len(peso_producto) == 0:
            self.result_area.configure(state="normal")
            self.result_area.insert(tk.END, 'No se encontró el producto con la categoría y SKU especificados.\n')
            self.result_area.configure(state="disabled")
            return

        peso_total = peso_producto[0] * num_bultos

        if peso_total >= 40:
            self.result_area.configure(state='normal')
            self.result_area.delete(1.0, tk.END)
            self.result_area.insert(tk.END, "El peso total excede los 40 kg.\nPor favor, cambie al tipo de producto 'Normal'.\n")
            self.result_area.configure(state='disabled')
            return

        self.peso_total_calculado = peso_total
        self.provincia_destino = province

        # Normalización del nombre de la provincia para asegurar la coincidencia
        province_normalized = normalizar_provincia(province)

        # Obtener tarifas
        tarifa_base_mrw, recargo_bultos_mrw, tarifa_total_mrw = obtener_tarifa_mrw(self.df_mrw, province_normalized, peso_total, num_bultos)
        tarifa_ontime_xs = obtener_tarifa_ontime_xs(self.df_ontime_xs, province_normalized, peso_total, modalidad)

        result = f'Para {num_bultos} bultos con SKU {sku} con peso total de {peso_total}kg y destino {province}:\n'
        
        if not np.isnan(tarifa_total_mrw):
            result += (f'Tarifa base MRW: {tarifa_base_mrw:.2f}€\n'
                    f'Recargo por bultos extra MRW: {recargo_bultos_mrw:.2f}€\n'
                    f'Total tarifa MRW con recargos: {tarifa_total_mrw:.2f}€\n')
        else:
            result += "Tarifa MRW: No disponible\n"

        if peso_total >= 10:
            tarifa_cbl = obtener_tarifa_cbl(self.df_cbl, province_normalized, peso_total)
            if not np.isnan(tarifa_cbl):
                recargo_combustible_cbl = tarifa_cbl * 0.035
                tarifa_cbl_total = tarifa_cbl + recargo_combustible_cbl
                result += (f'Tarifa base CBL: {tarifa_cbl:.2f}€\n'
                        f'Recargo por combustible CBL (3.5%): {recargo_combustible_cbl:.2f}€\n'
                        f'Total tarifa CBL con recargo: {tarifa_cbl_total:.2f}€\n')
            else:
                tarifa_cbl_total = np.nan
                result += "Tarifa CBL: No disponible\n"

        else:
            tarifa_cbl_total = np.nan

        if not np.isnan(tarifa_ontime_xs):
            recargo_combustible_ontime = tarifa_ontime_xs * 0.04
            recargo_seguro_ontime = tarifa_ontime_xs * 0.04
            tarifa_ontime_xs_total = tarifa_ontime_xs + recargo_combustible_ontime + recargo_seguro_ontime
            result += (f'Tarifa ONTIME XS ({modalidad} horas): {tarifa_ontime_xs:.2f}€\n'
                    f'Recargo por combustible ONTIME XS (4%): {recargo_combustible_ontime:.2f}€\n'
                    f'Recargo por seguro ONTIME XS (4%): {recargo_seguro_ontime:.2f}€\n'
                    f'Total tarifa ONTIME XS con recargos: {tarifa_ontime_xs_total:.2f}€\n')
        else:
            tarifa_ontime_xs_total = np.nan
            result += "Tarifa ONTIME XS: No disponible\n"

        tarifas = {
            "MRW": tarifa_total_mrw,
            "CBL": tarifa_cbl_total,
            "ONTIME XS": tarifa_ontime_xs_total
        }

        tarifas_validas = {k: v for k, v in tarifas.items() if not np.isnan(v)}
        if tarifas_validas:
            self.mejor_transportista = min(tarifas_validas, key=tarifas_validas.get)
            result += f'\nMejor transportista: {self.mejor_transportista} con tarifa {tarifas_validas[self.mejor_transportista]:.2f}€\n'
        else:
            result += "\nNo se encontró un transportista válido.\n"

        self.result_area.configure(state="normal")
        self.result_area.delete(1.0, tk.END)
        self.result_area.insert(tk.END, result)
        self.result_area.configure(state="disabled")


    def calculate_tariff_normal(self):
        palet_type = self.palet_type.get()
        product_height = float(self.height.get()) if self.height.get() else 0
        province = self.province.get().upper()

        if not palet_type or not product_height or not province:
            self.result_area.configure(state="normal")
            self.result_area.insert(tk.END, "Por favor, complete todos los campos.\n")
            self.result_area.configure(state="disabled")
            return

        zona = provincias_zonas.get(province)
        if not zona:
            self.result_area.configure(state="normal")
            self.result_area.insert(tk.END, f"Provincia {province} no reconocida.\n")
            self.result_area.configure(state="disabled")
            return

        base_area = 0.6 * 0.8 if palet_type == 'Medio Palet' else 1.2 * 0.8
        volume = base_area * (product_height + 0.15)
        kgs_cbl = volume * 200
        kgs_ontime = volume * 225

        print(f"Volumen calculado: {volume}, Peso CBL: {kgs_cbl}, Peso ONTIME: {kgs_ontime}")

        self.peso_total_calculado = kgs_cbl
        self.provincia_destino = province  # Guardar la provincia de destino
        
        tarifa_cbl = obtener_tarifa_cbl(self.df_cbl, province, kgs_cbl)
        tarifa_ontime = obtener_tarifa_ontime(self.df_ontime, zona, kgs_ontime)

        volumen_cercano = self.df_cbl[self.df_cbl['M3'] >= volume]['M3'].idxmin()
        if pd.isna(volumen_cercano):
            self.result_area.configure(state="normal")
            self.result_area.insert(tk.END, "No se encontró una tarifa CBL adecuada para el volumen calculado.\n")
            self.result_area.configure(state="disabled")
            return

        if np.isnan(tarifa_ontime):
            self.result_area.configure(state="normal")
            self.result_area.insert(tk.END, "No se encontró tarifa ONTIME válida.\n")
            self.result_area.configure(state="disabled")
            return

        recargo_combustible = tarifa_cbl * 0.035
        tarifa_cbl_total = tarifa_cbl + recargo_combustible

        recargo_combustible_ontime = tarifa_ontime * 0.04
        recargo_seguro_ontime = tarifa_ontime * 0.04
        tarifa_ontime_total = tarifa_ontime + recargo_combustible_ontime + recargo_seguro_ontime

        tarifas = {"CBL": tarifa_cbl_total, "ONTIME": tarifa_ontime_total}
        self.mejor_transportista = min(tarifas, key=tarifas.get) if tarifas else "Ninguno"

        result = (f"Para {palet_type} con altura {product_height}m y destino {province}:\n"
                f"Volumen: {volume:.2f} m³\n"
                f"KGS (CBL): {kgs_cbl:.2f} kg\n"
                f"KGS (ONTIME): {kgs_ontime:.2f} kg\n\n"
                f"Tarifa base CBL para {province}: {tarifa_cbl:.2f}€\n"
                f"Recargo por combustible CBL (3.5%): {recargo_combustible:.2f}€\n"
                f"Total tarifa CBL con recargo: {tarifa_cbl_total:.2f}€\n"
                f"Tarifa base ONTIME para {province}: {tarifa_ontime:.2f}€\n"
                f"Recargo por combustible ONTIME (4%): {recargo_combustible_ontime:.2f}€\n"
                f"Recargo por seguro ONTIME (4%): {recargo_seguro_ontime:.2f}€\n"
                f"Total tarifa ONTIME con recargos: {tarifa_ontime_total:.2f}€\n"
                f"\nMejor transportista: {self.mejor_transportista} con tarifa {tarifas[self.mejor_transportista]:.2f}€\n")

        self.result_area.configure(state="normal")
        self.result_area.delete(1.0, tk.END)
        self.result_area.insert(tk.END, result)
        self.result_area.configure(state="disabled")

    def calculate_return(self):
        # Crear una nueva ventana para seleccionar la provincia de origen de la devolución
        return_window = tk.Toplevel(self.master)
        return_window.title("Cálculo de Devoluciones")
        return_window.geometry("200x400")  # Ajustar el tamaño de la ventana

        padx_val = 10  # Espacio horizontal entre widgets
        pady_val = 5   # Espacio vertical entre widgets

        # Widgets en la ventana de devolución
        label_return_province = ttk.Label(return_window, text="Provincia de Origen:")
        label_return_province.pack(padx=padx_val, pady=pady_val, anchor='w')
        return_province = ttk.Combobox(return_window, values=list(provincias_zonas.keys()))
        return_province.pack(padx=padx_val, pady=pady_val, fill='x')

        label_product_type = ttk.Label(return_window, text="Tipo de Producto:")
        label_product_type.pack(padx=padx_val, pady=pady_val, anchor='w')
        product_type = ttk.Combobox(return_window, values=['Normal', 'XS'])
        product_type.pack(padx=padx_val, pady=pady_val, fill='x')
        product_type.bind("<<ComboboxSelected>>", lambda event: self.toggle_devol_widgets(product_type.get(), return_window))

        label_categoria = ttk.Label(return_window, text="Categoría del Producto:")
        label_categoria.pack(padx=padx_val, pady=pady_val, anchor='w')
        self.categoria_devol = ttk.Combobox(return_window, values=list(self.df_productos_devol['CATEGORIAS'].unique()))
        self.categoria_devol.pack(padx=padx_val, pady=pady_val, fill='x')

        label_sku = ttk.Label(return_window, text="SKU del Producto:")
        label_sku.pack(padx=padx_val, pady=pady_val, anchor='w')
        self.sku_devol = ttk.Combobox(return_window, values=list(self.df_productos_devol['SKU'].unique()))
        self.sku_devol.pack(padx=padx_val, pady=pady_val, fill='x')

        label_num_bultos = ttk.Label(return_window, text="Número de Bultos:")
        label_num_bultos.pack(padx=padx_val, pady=pady_val, anchor='w')
        self.num_bultos_devol = ttk.Entry(return_window)
        self.num_bultos_devol.pack(padx=padx_val, pady=pady_val, fill='x')

        # Botón para calcular la devolución
        calculate_button = ttk.Button(
            return_window, 
            text="Calcular Devolución", 
            command=lambda: self.calculate_return_tariff(
                return_province.get(), 
                product_type.get(), 
                self.categoria_devol.get(), 
                self.sku_devol.get(), 
                self.num_bultos_devol.get(), 
                return_window
            )
        )
        calculate_button.pack(pady=pady_val)

    def toggle_devol_widgets(self, tipo_producto, window):
        if tipo_producto == 'XS':
            self.categoria_devol.configure(state='normal')
            self.sku_devol.configure(state='normal')
            self.num_bultos_devol.configure(state='normal')
        else:
            self.categoria_devol.configure(state='disabled')
            self.sku_devol.configure(state='disabled')
            self.num_bultos_devol.configure(state='disabled')    
        
        # Función para buscar el peso en devoluciones
    def obtener_peso_producto_xs(self, categoria, sku, num_bultos):
        # Buscar el peso del producto en el DataFrame específico de devoluciones
        peso_producto = self.df_productos_devol[(self.df_productos_devol['CATEGORIAS'] == categoria) & (self.df_productos_devol['SKU'] == sku)]['PESO BRUTO (kg)'].values
        if len(peso_producto) == 0:
            messagebox.showerror('Error', 'No se encontró el producto con la categoría y el SKU indicado')
            return np.nan
        
        peso_total = peso_producto[0] * int(num_bultos)
        return peso_total

    def calculate_return_tariff(self, province, tipo_producto, categoria, sku, num_bultos, window):
        # Verificación básica
        if not province or not tipo_producto or (tipo_producto == 'XS' and (not categoria or not sku or not num_bultos)):
            messagebox.showerror("Error", "Por favor, complete todos los campos.")
            return

        # Normalización de la provincia
        province_normalized = normalizar_provincia(province)

        # Configurar los valores de cálculo para devoluciones
        destino = 'VALENCIA'
        tarifa_cbl_total = np.nan
        tarifa_ontime_total = np.nan

        if tipo_producto == 'Normal':
            # Siempre se considera un Palet Completo para devoluciones normales
            base_area = 1.2 * 0.8
            altura_producto = 2.0  # Asumimos una altura máxima para el palet
            volume = base_area * (altura_producto + 0.15)
            peso_total_cbl = volume * 200
            peso_total_ontime = volume * 225

            # Calcular tarifa CBL con recargos
            tarifa_cbl = obtener_tarifa_cbl(self.df_cbl, province_normalized, peso_total_cbl)
            if not np.isnan(tarifa_cbl):
                recargo_combustible_cbl = tarifa_cbl * 0.035
                recargo_devolucion = tarifa_cbl * 0.2
                tarifa_cbl_total = tarifa_cbl + recargo_combustible_cbl + recargo_devolucion

            # Calcular tarifa ONTIME con recargos
            tarifa_ontime = obtener_tarifa_ontime(self.df_ontime, province_normalized, peso_total_ontime)
            if not np.isnan(tarifa_ontime):
                recargo_combustible_ontime = tarifa_ontime * 0.04
                recargo_seguro_ontime = tarifa_ontime * 0.04
                tarifa_ontime_total = tarifa_ontime + recargo_combustible_ontime + recargo_seguro_ontime

        elif tipo_producto == 'XS':
            # Calcular peso del producto XS
            peso_producto = self.df_productos_devol[
                (self.df_productos_devol['CATEGORIAS'] == categoria) &
                (self.df_productos_devol['SKU'] == sku)
            ]['PESO BRUTO (kg)'].values

            if len(peso_producto) == 0:
                messagebox.showerror("Error", "Producto no encontrado.")
                return

            peso_total = peso_producto[0] * int(num_bultos)

            # Calcular tarifa CBL con recargos
            tarifa_cbl = obtener_tarifa_cbl(self.df_cbl, province_normalized, peso_total)
            if not np.isnan(tarifa_cbl):
                recargo_combustible_cbl = tarifa_cbl * 0.035
                recargo_devolucion = tarifa_cbl * 0.2
                tarifa_cbl_total = tarifa_cbl + recargo_combustible_cbl + recargo_devolucion

            # Calcular tarifa ONTIME XS con recargos
            tarifa_ontime_xs = obtener_tarifa_ontime_xs(self.df_ontime_xs, province_normalized, peso_total, '24')
            if not np.isnan(tarifa_ontime_xs):
                recargo_combustible_ontime = tarifa_ontime_xs * 0.04
                recargo_seguro_ontime = tarifa_ontime_xs * 0.04
                tarifa_ontime_total = tarifa_ontime_xs + recargo_combustible_ontime + recargo_seguro_ontime

        # Comparar tarifas y mostrar el mejor resultado
        tarifas = {'CBL': tarifa_cbl_total, 'ONTIME': tarifa_ontime_total}
        tarifas_validas = {k: v for k, v in tarifas.items() if not np.isnan(v)}

        if not tarifas_validas:
            messagebox.showerror('Error', 'No se encontraron tarifas válidas para la devolución.')
            return

        mejor_transportista = min(tarifas_validas, key=tarifas_validas.get)

        # Mostrar el resultado
        result = (f'Para devolución desde {province} a Valencia con producto tipo {tipo_producto}:\n'
                f'Tarifa CBL: {tarifa_cbl_total:.2f}€\n'
                f'Recargo de CBL por combustible (3.5%): {recargo_combustible_cbl:.2f}€\n'
                f'Recargo de CBL por devolución (20%): {recargo_devolucion:.2f}€\n'
                f'Tarifa total CBL: {tarifa_cbl_total:.2f}€\n'
                f'Tarifa ONTIME: {tarifa_ontime_total:.2f}€\n'
                f'Recargo de ONTIME por combustible (4%): {recargo_combustible_ontime:.2f}€\n'
                f'Recargo de ONTIME por seguro (4%): {recargo_seguro_ontime:.2f}€\n'
                f'Tarifa total ONTIME: {tarifa_ontime_total:.2f}€\n')

        self.result_area.configure(state='normal')
        self.result_area.delete(1.0, tk.END)
        self.result_area.insert(tk.END, result)
        self.result_area.configure(state='disabled')

        window.destroy()  # Cerrar la ventana de devolución


    def obtener_peso_producto_xs(self, categoria, sku, num_bultos):
        # Buscar el peso del producto en el DataFrame específico de devoluciones
        peso_producto = self.df_productos_devol[(self.df_productos_devol['CATEGORIAS'] == categoria) & (self.df_productos_devol['SKU'] == sku)]['PESO BRUTO (kg)'].values
        if len(peso_producto) == 0:
            messagebox.showerror('Error', 'No se encontró el producto con la categoría y el SKU indicado')
            return np.nan
        
        peso_total = peso_producto[0] * int(num_bultos)
        return peso_total

   
    def show_map(self):
        if not hasattr(self, 'peso_total_calculado') or self.peso_total_calculado is None:
            self.result_area.configure(state="normal")
            self.result_area.insert(tk.END, "\nPor favor, calcule la tarifa primero.\n")
            self.result_area.configure(state="disabled")
            return

        peso_total = self.peso_total_calculado
            
        shapefile_path = 'C:/Users/Svan/OneDrive - SVAN TRADING SL/Documentos/Proyectos/Calculadora transporte/lineas_limite/SHP_ETRS89/ll_provinciales_inspire_peninbal_etrs89/ll_provinciales_inspire_peninbal_etrs89.shp'            
        gdf = gpd.read_file(shapefile_path)
        gdf['PROVINCIA'] = gdf['NAME_BOUND'].str.split('#').str[0].str.upper()

        # Ajustar nombres de provincias en gdf
        gdf = ajustar_nombres_provincias(gdf, 'PROVINCIA')

        # Normalizar los nombres de las provincias
        gdf['PROVINCIA'] = gdf['PROVINCIA'].apply(normalizar_provincia)

        # Normalizar el nombre de la provincia de destino
        provincia_destino_normalizada = normalizar_provincia(self.provincia_destino)

        # Definir los colores para los diferentes transportistas
        color_mapping = {"CBL": "red", "ONTIME": "blue", "ONTIME XS": "skyblue", "MRW": "yellow"}
            
        # Aplicar el color según el mejor transportista solo a las provincias de origen y destino
        gdf['COLOR'] = 'lightgrey'  # Colorear el resto de las provincias en gris claro
        gdf.loc[gdf['PROVINCIA'] == 'VALENCIA', 'COLOR'] = color_mapping[self.mejor_transportista]
        gdf.loc[gdf['PROVINCIA'] == provincia_destino_normalizada, 'COLOR'] = color_mapping[self.mejor_transportista]

        new_window = tk.Toplevel(self.master)
        new_window.title("Mapa de Transportistas")

        fig, ax = plt.subplots(1, 1, figsize=(10, 6))

        gdf.plot(ax=ax, color=gdf['COLOR'], edgecolor="black")

        plt.title(f"Mejor Transportista para envío desde Valencia a {self.provincia_destino}")

        canvas = FigureCanvasTkAgg(fig, master=new_window)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        plt.show()

In [ ]:
# Aplicación principal para lanzar la lógica
def start_app():
    root = tk.Tk()
    app = TransportCalculatorApp(root)
    root.mainloop()

if __name__ == '__main__':
    start_app()

Buscando tarifa CBL para zona: 'A CORUNA', peso: 12.6
Tarifa CBL encontrada para A CORUNA: 9.62
Buscando tarifa ONTIME XS para zona: 'A CORUNA', peso: 12.6, modalidad: 24
Tarifa ONTIME XS encontrada: 7.09
Buscando tarifa CBL para zona: 'MURCIA', peso: 412.8
Tarifa CBL encontrada para MURCIA: 56.53
Buscando tarifa ONTIME para zona: 'MURCIA', peso: 464.40000000000003
Tarifa ONTIME encontrada: 51.19893254047619
